In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import os
import seaborn as sns
from glob import glob
import warnings
from autopreprocess_pipeline import *
from autopreprocessing import dataset as ds
import shutil 
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV, cross_val_score
from keras.layers import Dense, Flatten, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRFClassifier 
import cupy as cp
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

2024-06-25 22:22:10.151466: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 22:22:10.237279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
X_val = np.load("X_val.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")
y_val = np.load("y_val.npy")

In [20]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1]*X_val.shape[2])

In [21]:
# Assuming X_train and y_train are your training data and labels
# Combine X_train and y_train to shuffle them together
combined = np.c_[X_train.reshape(len(X_train), -1), y_train.reshape(len(y_train), -1)]
np.random.shuffle(combined)

# Separate the combined array back into X_train and y_train
X_train_shuffled = combined[:, :X_train.shape[1]]
y_train_shuffled = combined[:, X_train.shape[1]:].reshape(-1)

# Assuming X_test and y_test are your test data and labels
# Combine X_test and y_test to shuffle them together
combined_test = np.c_[X_test.reshape(len(X_test), -1), y_test.reshape(len(y_test), -1)]
np.random.shuffle(combined_test)

# Separate the combined array back into X_test and y_test
X_test_shuffled = combined_test[:, :X_test.shape[1]]
y_test_shuffled = combined_test[:, X_test.shape[1]:].reshape(-1)

### Feature importance indices - AdaBoost (base estimator: DecisionTreeClassifier)

In [27]:
param_grid_adaboost = {
    'n_estimators': [30,60,90],
    'learning_rate' : [0.1,0.4,1],
    'algorithm' : ["SAMME"],
    'random_state' : [42]
}
ada = AdaBoostClassifier()
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_adaboost, cv = 5, scoring='accuracy')

grid_search_ada = grid_search_ada.fit(X_train_shuffled,y_train_shuffled)

best_model_ada = grid_search_ada.best_estimator_
best_score_ada = grid_search_ada.best_score_
best_params_ada = grid_search_ada.best_params_

In [28]:
feature_importances = best_model_ada.feature_importances_ 
print(f'Best Ada accuracy: {best_score_ada}, params: {best_params_ada}')

Best Ada accuracy: 0.9276923076923078, params: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 90, 'random_state': 42}


In [36]:
ada = AdaBoostClassifier(algorithm="SAMME", learning_rate = 1, n_estimators=90, random_state = 42)

In [ ]:
cval_ada = cross_val_score(ada,X_train_shuffled,y_train_shuffled)

In [ ]:
print(cval_ada)

### Feature importance indices - AdaBoost in Recursive Feature Elimination 

In [32]:
rfecv = RFECV(estimator=ada,cv=5, verbose = 2, n_jobs = -1)

In [33]:
rfecv.fit(X_train_shuffled,y_train_shuffled)

KeyboardInterrupt: 